In [1]:
import pandas as pd

import pandas as pd
import urllib.request
from pathlib import Path
import os
import csv
import tqdm

In [2]:
# ========= Scraping and Analysing Amtrack Delay Times =========

# Scraping
# Collecting Amtrack station IDs

id_url = "https://juckins.net/amtrak_status/archive/html/stations.php"
req_id = urllib.request.Request(id_url, headers={'User-Agent': 'Mozilla/5.0'})
id_html = urllib.request.urlopen(req_id).read()

station_ids = pd.read_html(id_html, header = 1)[0]

print(station_ids)
len(station_ids)

    Code                   Name
0    ABE           Aberdeen, MD
1    ABQ        Albuquerque, NM
2    ACA  Antioch-Pittsburg, CA
3    ACD            Arcadia, MO
4    ADM            Ardmore, OK
..   ...                    ...
552  XXX                    NaN
553  YAZ         Yazoo City, MS
554  YEM           Yemassee, SC
555  YNY            Yonkers, NY
556  YUM               Yuma, AZ

[557 rows x 2 columns]


557

In [3]:

# Translating station Codes into a list of usable URLs

station_urls = {}
years_to_use = [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]

generic_url = "https://juckins.net/amtrak_status/archive/html/history.php?train_num=all&station={station}&date_start=01%2F01%2F{low_year}&date_end=12%2F31%2F{high_year}&df1=1&df2=1&df3=1&df4=1&df5=1&df6=1&df7=1&sort=schDp&sort_dir=DESC&co=gt&limit_mins=&dfon=1"

station_code = station_ids['Code'].tolist()

for Code in station_code:
    for year in years_to_use:
        station_url = generic_url.format(station = Code, low_year = str(year), high_year = str(year + 1))
        url_Name = Code + "_" + str(year) + "-" + str(year + 1)
        station_urls.update({url_Name : station_url})

len(station_urls)

6684

In [4]:
# Scraping All Data Pages

# Making a dictionary we can strip values out of
stationDic = station_urls.copy()

# Getting a list of already downloaded files
collectedTrainData = os.listdir(path='/Users/NorthCarpenter/Documents/Sort/Employment/Making Employment/Train Code/trainDataStation')
collectedTrainData.remove("noDataStatYr.csv")
collectedTrainData.remove(".DS_Store")

# Getting the list of files without data
with open('/Users/NorthCarpenter/Documents/Sort/Employment/Making Employment/Train Code/trainDataStation/noDataStatYr.csv', 'r', newline = '') as noDatCSV:
    reader = csv.reader(noDatCSV)
    noDataList = list(reader)

# Remove URL's with no data or already downloaded

if len(collectedTrainData) >= 1:
    for stationYear in collectedTrainData:
        stationDic.pop(stationYear[:-4])

if len(noDataList) >= 1:
    for stationYear in noDataList:
        stationDic.pop(stationYear[0])

# Set Path of folder to save data
pathTrainFolder = os.path.abspath('/Users/NorthCarpenter/Documents/Sort/Employment/Making Employment/Train Code/trainDataStation')


for url in tqdm.tqdm(stationDic):
    try:
        req_station = urllib.request.Request(stationDic[url], headers={'User-Agent': 'Mozilla/5.0'})
        station_html = urllib.request.urlopen(req_station).read()
        station_data = pd.read_html(station_html, header = 1)
        station_data = station_data[0]
        outputPath = os.path.join(pathTrainFolder, url + '.csv')
        station_data.to_csv(outputPath, index = False)
        print(url)
    except:
        with open('/Users/NorthCarpenter/Documents/Sort/Employment/Making Employment/Train Code/trainDataStation/noDataStatYr.csv', 'a', newline='') as csv_file:
            csv_file.write(url)
            csv_file.write('\n')
        print(url, " FAIL")

0it [00:00, ?it/s]
